Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from yahoo_query import *

In [14]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
stock_list = list(set(tickers[3:] + stock_list))

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
    earnings_info['earnBeatsBefore'] = 0
    earnings_info['earnMissBefore'] = 0

    for idx, row in earnings_info.iterrows():
        earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
        earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
    earnings_info = earnings_info.shift(1)


    earnings_moves = past_earnings(ticker).sort_index()
    earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
                                    (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


    earnings_df = pd.concat([earnings_info.reset_index(), 
                             earnings_moves.reset_index()], axis = 1)
    earnings_df.columns = ['quarter' if col == 'index' else col for col in earnings_df.columns.tolist()]
    earnings_df['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_df, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [15]:
earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
earnings.to_csv('full_data.csv')
ks.to_csv('keyStats_v3.csv')

Failed: PRLB
0.03% Completed
1 total failures
Failed: MTDR
0.05% Completed
2 total failures
Failed: CNX
0.08% Completed
3 total failures
Failed: XOP
0.11% Completed
4 total failures
Failed: ACUR
0.13% Completed
5 total failures
Failed: PXH
0.16% Completed
6 total failures
Failed: SALT
0.19% Completed
7 total failures
Failed: WCG
0.22% Completed
8 total failures
Failed: VEDL
0.24% Completed
9 total failures
Failed: TSCO
0.27% Completed
10 total failures
Failed: SPNC
0.30% Completed
11 total failures
Failed: CARB
0.32% Completed
12 total failures
Failed: TM
0.35% Completed
13 total failures
Failed: FUEL
0.38% Completed
14 total failures
Failed: BAL
0.40% Completed
15 total failures
Failed: CNHI
0.43% Completed
16 total failures
Failed: AVD
0.46% Completed
17 total failures
Failed: IART
0.48% Completed
18 total failures
Failed: DXLG
0.51% Completed
19 total failures
Failed: MLM
0.54% Completed
20 total failures
Failed: UONEK
0.57% Completed
21 total failures
Failed: ZBH
0.59% Completed
22

Failed: BKU
4.74% Completed
176 total failures
Failed: MGA
4.76% Completed
177 total failures
Failed: EHTH
4.79% Completed
178 total failures
Failed: VO
4.82% Completed
179 total failures
Failed: FDC
4.85% Completed
180 total failures
Failed: TTS
4.87% Completed
181 total failures
Failed: EXR
4.90% Completed
182 total failures
Failed: RMBS
4.93% Completed
183 total failures
Failed: ANDV
4.95% Completed
184 total failures
Failed: SPXC
4.98% Completed
185 total failures
Failed: AMBR
5.01% Completed
186 total failures
Failed: USMV
5.03% Completed
187 total failures
Failed: VTWO
5.06% Completed
188 total failures
Failed: ITCI
5.09% Completed
189 total failures
Failed: CRSP
5.11% Completed
190 total failures
Failed: UFPI
5.14% Completed
191 total failures
Failed: EVEP
5.17% Completed
192 total failures
Failed: CCO
5.20% Completed
193 total failures
Failed: FIG
5.22% Completed
194 total failures
Failed: EWT
5.25% Completed
195 total failures
Failed: FAZ
5.28% Completed
196 total failures
Fai

Failed: DPK
9.39% Completed
349 total failures
Failed: WK
9.42% Completed
350 total failures
Failed: BWX
9.45% Completed
351 total failures
Failed: UCTT
9.48% Completed
352 total failures
Failed: RARE
9.50% Completed
353 total failures
Failed: UMPQ
9.53% Completed
354 total failures
Failed: ORN
9.56% Completed
355 total failures
Failed: EUSC
9.58% Completed
356 total failures
Failed: OKTA
9.61% Completed
357 total failures
Failed: DBJP
9.64% Completed
358 total failures
Failed: MGLN
9.66% Completed
359 total failures
Failed: MAS
9.69% Completed
360 total failures
Failed: MGK
9.72% Completed
361 total failures
Failed: DRI
9.74% Completed
362 total failures
Failed: POL
9.77% Completed
363 total failures
Failed: TA
9.80% Completed
364 total failures
Failed: GPN
9.83% Completed
365 total failures
Failed: BKEP
9.85% Completed
366 total failures
Failed: DGS
9.88% Completed
367 total failures
Failed: NSC
9.91% Completed
368 total failures
Failed: USM
9.93% Completed
369 total failures
Failed:

Failed: DCI
13.97% Completed
519 total failures
Failed: TAN
14.00% Completed
520 total failures
Failed: KEYS
14.02% Completed
521 total failures
Failed: POR
14.05% Completed
522 total failures
Failed: OKE
14.08% Completed
523 total failures
Failed: REG
14.10% Completed
524 total failures
Failed: SGG
14.13% Completed
525 total failures
Failed: GYLD
14.16% Completed
526 total failures
Failed: BKLN
14.19% Completed
527 total failures
Failed: RTH
14.21% Completed
528 total failures
Failed: IPXL
14.24% Completed
529 total failures
Failed: INCY
14.27% Completed
530 total failures
Failed: AXGN
14.29% Completed
531 total failures
Failed: EDR
14.32% Completed
532 total failures
Failed: ATLC
14.35% Completed
533 total failures
Failed: OEX
14.37% Completed
534 total failures
Failed: AMRS
14.40% Completed
535 total failures
Failed: BBBY
14.43% Completed
536 total failures
Failed: FHCO
14.45% Completed
537 total failures
Failed: PSP
14.48% Completed
538 total failures
Failed: PDCO
14.51% Completed


Failed: TOWR
18.55% Completed
689 total failures
Failed: HMLP
18.57% Completed
690 total failures
Failed: HOLI
18.60% Completed
691 total failures
Failed: AXP
18.63% Completed
692 total failures
Failed: SF
18.65% Completed
693 total failures
Failed: MU
18.68% Completed
694 total failures
Failed: SVXY
18.71% Completed
695 total failures
Failed: IYW
18.73% Completed
696 total failures
Failed: CHL
18.76% Completed
697 total failures
Failed: GRFS
18.79% Completed
698 total failures
Failed: KEY
18.82% Completed
699 total failures
Failed: WEAT
18.84% Completed
700 total failures
Failed: SGMO
18.87% Completed
701 total failures
Failed: PAH
18.90% Completed
702 total failures
Failed: MKC
18.92% Completed
703 total failures
Failed: UFS
18.95% Completed
704 total failures
Failed: NVO
18.98% Completed
705 total failures
Failed: OSUR
19.00% Completed
706 total failures
Failed: TRIB
19.03% Completed
707 total failures
Failed: ESS
19.06% Completed
708 total failures
Failed: RE
19.08% Completed
709 t

...

Failed: ZS
85.76% Completed
3186 total failures
Failed: WPZ
85.79% Completed
3187 total failures
Failed: DWX
85.81% Completed
3188 total failures
Failed: CONN
85.84% Completed
3189 total failures
Failed: ETE
85.87% Completed
3190 total failures
Failed: EDAP
85.90% Completed
3191 total failures
Failed: HASI
85.92% Completed
3192 total failures
Failed: VECO
85.95% Completed
3193 total failures
Failed: ERI
85.98% Completed
3194 total failures
Failed: SNA
86.00% Completed
3195 total failures
Failed: CAPL
86.03% Completed
3196 total failures
Failed: JPNL
86.06% Completed
3197 total failures
Failed: ESRT
86.08% Completed
3198 total failures
Failed: RLI
86.11% Completed
3199 total failures
Failed: CHKP
86.14% Completed
3200 total failures
Failed: TRST
86.16% Completed
3201 total failures
Failed: FNDF
86.19% Completed
3202 total failures
Failed: GSL
86.22% Completed
3203 total failures
Failed: IAU
86.24% Completed
3204 total failures
Failed: BHE
86.27% Completed
3205 total failures
Failed: GVA

Failed: CYTK
90.26% Completed
3353 total failures
Failed: EDU
90.28% Completed
3354 total failures
Failed: ARNC
90.31% Completed
3355 total failures
Failed: PBPB
90.34% Completed
3356 total failures
Failed: NAVI
90.36% Completed
3357 total failures
Failed: SPGI
90.39% Completed
3358 total failures
Failed: VLY
90.42% Completed
3359 total failures
Failed: HTLD
90.44% Completed
3360 total failures
Failed: JELD
90.47% Completed
3361 total failures
Failed: ALXN
90.50% Completed
3362 total failures
Failed: KMT
90.52% Completed
3363 total failures
Failed: SHOO
90.55% Completed
3364 total failures
Failed: AMN
90.58% Completed
3365 total failures
Failed: PNQI
90.61% Completed
3366 total failures
Failed: LAMR
90.63% Completed
3367 total failures
Failed: RAVN
90.66% Completed
3368 total failures
Failed: MOS
90.69% Completed
3369 total failures
Failed: CENTA
90.71% Completed
3370 total failures
Failed: CNC
90.74% Completed
3371 total failures
Failed: NTWK
90.77% Completed
3372 total failures
Faile

Failed: RWR
94.72% Completed
3519 total failures
Failed: DCP
94.75% Completed
3520 total failures
Failed: MXI
94.78% Completed
3521 total failures
Failed: HA
94.80% Completed
3522 total failures
Failed: MFA
94.83% Completed
3523 total failures
Failed: SWKS
94.86% Completed
3524 total failures
Failed: BPT
94.89% Completed
3525 total failures
Failed: FCSC
94.91% Completed
3526 total failures
Failed: KIM
94.94% Completed
3527 total failures
Failed: ASH
94.97% Completed
3528 total failures
Failed: FMC
94.99% Completed
3529 total failures
Failed: ADP
95.02% Completed
3530 total failures
Failed: WLKP
95.05% Completed
3531 total failures
Failed: THS
95.07% Completed
3532 total failures
Failed: NRZ
95.10% Completed
3533 total failures
Failed: OSIR
95.13% Completed
3534 total failures
Failed: HGGGQ
95.15% Completed
3535 total failures
Failed: GLFMQ
95.18% Completed
3536 total failures
Failed: EWS
95.21% Completed
3537 total failures
Failed: CNK
95.24% Completed
3538 total failures
Failed: OUT
9

Failed: AERI
99.19% Completed
3685 total failures
Failed: KAI
99.22% Completed
3686 total failures
Failed: DRD
99.25% Completed
3687 total failures
Failed: AMCX
99.27% Completed
3688 total failures
Failed: RPV
99.30% Completed
3689 total failures
Failed: AAPL
99.33% Completed
3690 total failures
Failed: URA
99.35% Completed
3691 total failures
Failed: HMHC
99.38% Completed
3692 total failures
Failed: UVE
99.41% Completed
3693 total failures
Failed: TXT
99.43% Completed
3694 total failures
Failed: GILD
99.46% Completed
3695 total failures
Failed: IYT
99.49% Completed
3696 total failures
Failed: OOMA
99.52% Completed
3697 total failures
Failed: CRK
99.54% Completed
3698 total failures
Failed: YRCW
99.57% Completed
3699 total failures
Failed: BRS
99.60% Completed
3700 total failures
Failed: WINT
99.62% Completed
3701 total failures
Failed: MXEF
99.65% Completed
3702 total failures
Failed: FMX
99.68% Completed
3703 total failures
Failed: K
99.70% Completed
3704 total failures
Failed: LPI
9

ValueError: No objects to concatenate

In [ ]:
ks = pd.read_csv('keyStats.csv',index_col = 0).drop_duplicates()
earnings = pd.read_csv('earningsHist_unshifted.csv', index_col = 0).drop_duplicates()

In [ ]:
ks = ks.drop(ks.T.isnull().sum(axis = 1).sort_values(ascending = False).head(18).index.tolist(),
             axis = 1)

score_cols = ['52WeekChange', 'earningsGrowth',
              'earningsQuarterlyGrowth', 'forwardPE',
              'grossMargins', 'grossProfits',
              'pegRatio', 'profitMargins', 
              'returnOnAssets',
              'returnOnEquity', 'revenueGrowth', 'sector']




earnings.quarter = pd.to_datetime(earnings.quarter)
earnings = earnings.groupby(['Underlying','quarter']).sum().reset_index()
ticker = 'A'
latest_earnings = []
for ticker in earnings.Underlying.drop_duplicates().tolist():
    latest_earnings.append(earnings[earnings['Underlying'] == ticker][earnings.quarter == max(earnings[earnings['Underlying'] == ticker].quarter)])
latest_earnings = pd.concat(latest_earnings, axis = 0)
latest_earnings.index = latest_earnings.Underlying

earn_cols = ['quarter', '1Year', '1month', '3month', '6month','totalLiab',
             'totalStockholderEquity','netIncome',
             'operatingIncome',
             'totalRevenue','dividendsPaid',
             'investments',
             'longTermInvestments',
             'shortTermInvestments']

latest_earnings = latest_earnings[earn_cols]


ks = ks[score_cols]

names = ks.join(latest_earnings).drop_duplicates()

names['dividendsPaid'] = -names['dividendsPaid']

names['payoutRatio'] = names.dividendsPaid/names.netIncome
names['debtEquityRatio'] = names.totalLiab/names.totalStockholderEquity
names['netMargin'] = names.netIncome/names.totalRevenue
names['roic'] = names.operatingIncome/(names.longTermInvestments + names.shortTermInvestments)

names = names[['debtEquityRatio',
       '52WeekChange','earningsGrowth',
       'earningsQuarterlyGrowth', 'forwardPE',
       'grossMargins', 'grossProfits',
       'pegRatio', 'profitMargins', 
       'returnOnAssets',
       'returnOnEquity', 'revenueGrowth', 'sector']]

names = names[names['52WeekChange'] != 'Infinity']
names['52WeekChange'] = pd.to_numeric(names['52WeekChange'])

removes = pd.DataFrame(names.isnull().sum(axis = 1).sort_values(ascending = False))
removes = removes[removes[0] <= 3]

names = names[names.index.isin(removes.index.tolist())].fillna(0)

names_mean = names.groupby('sector').mean()
names_mean = names[['sector']].join(names_mean, on = 'sector')

names_high = names.groupby('sector').max()
names_high = names[['sector']].join(names_high, on = 'sector')

names_low = names.groupby('sector').min()
names_low = names[['sector']].join(names_low, on = 'sector')

names_std = names.groupby('sector').std()
names_std = names[['sector']].join(names_std, on = 'sector')

del names['sector'], names_mean['sector'], names_high['sector'], names_low['sector'], names_std['sector']

score = (names - names_mean)/names_std#(names_high - names_low)

del score['forwardPE']

score['score'] = ((score['52WeekChange'])/3 -
                   (score['debtEquityRatio'] + score['pegRatio'])/3 +
                   (score['earningsGrowth'] + score['revenueGrowth'])/3)
score.join(ks[['sector']]).sort_values('score',ascending = False).to_csv('leaps_scores.csv')